In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

# 导入我们自己写的 train_mlp_numpy.py 里的函数和类
# 确保所有 .py 文件和这个 notebook 在同一个目录下
from train_mlp_numpy import train, accuracy
from mlp_numpy import MLP
from modules import *

print("所有模块导入成功！")
def prepare_data(random_seed):
    """
    生成、预处理并分割 moons 数据集。
    """
    # 1. 生成 moons 数据集
    X, y_scalar = make_moons(n_samples=1000, noise=0.1, random_state=random_seed)

    # 2. 标签转为 One-Hot 编码
    y_one_hot = np.zeros((len(y_scalar), 2))
    y_one_hot[np.arange(len(y_scalar)), y_scalar] = 1

    # 3. 分割训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=random_seed)

    # 4. 将数据打包
    return (X_train, y_train, X_test, y_test)

print("数据准备函数定义完成。")
def run_single_experiment(use_leaky_relu, random_seed):
    """
    运行一次完整的训练实验，并判断是否收敛。
    
    Args:
        use_leaky_relu (bool): 是否使用 Leaky ReLU。
        random_seed (int): 用于数据生成和权重初始化的随机种子。
        
    Returns:
        bool: 如果模型在测试集上的最终准确率达到95%，则返回 True，否则返回 False。
    """
    # 1. 准备数据
    data_tuple = prepare_data(random_seed)
    
    # 2. 定义超参数
    DNN_HIDDEN_UNITS = '20,20'  
    LEARNING_RATE = 1
    MAX_EPOCHS = 1000 
    EVAL_FREQ = 100 

    # 3. 调用我们修改过的 train 函数
    # 注意：我们不需要接收返回的日志，只需要最后一个返回值 mlp
    _, _, _, _, mlp = train(
        data=data_tuple,
        dnn_hidden_units=DNN_HIDDEN_UNITS,
        learning_rate=LEARNING_RATE,
        max_steps=MAX_EPOCHS,
        eval_freq=EVAL_FREQ,
        batch_size='full', 
        momentum=0,
        leaky=use_leaky_relu 
    )
    
    # 4. 在实验结束后，评估最终模型的性能
    test_x, test_y = data_tuple[2], data_tuple[3]
    softmax_layer = SoftMax() # 需要一个softmax层来转换输出
    
    test_logits = mlp.forward(test_x)
    test_probs = softmax_layer.forward(test_logits)
    final_accuracy = accuracy(test_probs, test_y)
    
    # 5. 判断是否收敛
    # 我们定义“收敛”为最终测试准确率达到一个较高的阈值，比如 95%
    convergence_threshold = 0.95
    
    return final_accuracy >= convergence_threshold

print("单次实验函数定义完成。")
# 实验参数
num_experiments = 200

# 用于存储每次实验的结果
relu_convergences = 0
leaky_relu_convergences = 0

print("开始运行标准 ReLU 的 200 次实验...")
for i in range(num_experiments):
    # 使用不同的随机种子 (i) 来保证每次实验的独立性
    if run_single_experiment(use_leaky_relu=False, random_seed=i):
        relu_convergences += 1

print("\n开始运行 Leaky ReLU 的 200 次实验...")
for i in range(num_experiments):
    if run_single_experiment(use_leaky_relu=True, random_seed=i):
        leaky_relu_convergences += 1

print("\n--- 实验结果统计 ---")
print(f"标准 ReLU: 在 {num_experiments} 次实验中，成功收敛 {relu_convergences} 次。")
print(f"Leaky ReLU: 在 {num_experiments} 次实验中，成功收敛 {leaky_relu_convergences} 次。")

# 计算并打印收敛率
relu_rate = (relu_convergences / num_experiments) * 100
leaky_relu_rate = (leaky_relu_convergences / num_experiments) * 100

print(f"\n收敛率对比:")
print(f"标准 ReLU: {relu_rate:.2f}%")
print(f"Leaky ReLU: {leaky_relu_rate:.2f}%")

所有模块导入成功！
数据准备函数定义完成。
单次实验函数定义完成。
开始运行标准 ReLU 的 200 次实验...
Full batch gradient descent
Epoch: 0, Test Loss: 0.6940, Test Accuracy: 0.4600, Train Accuracy: 0.5100
Epoch: 100, Test Loss: 0.6465, Test Accuracy: 0.7750, Train Accuracy: 0.8063
Epoch: 200, Test Loss: 0.3186, Test Accuracy: 0.8550, Train Accuracy: 0.8562
Epoch: 300, Test Loss: 0.2157, Test Accuracy: 0.8800, Train Accuracy: 0.9025
Epoch: 400, Test Loss: 0.2027, Test Accuracy: 0.8950, Train Accuracy: 0.9087
Epoch: 500, Test Loss: 0.2000, Test Accuracy: 0.9050, Train Accuracy: 0.9075
Epoch: 600, Test Loss: 0.1985, Test Accuracy: 0.9100, Train Accuracy: 0.9100
Epoch: 700, Test Loss: 0.1981, Test Accuracy: 0.9050, Train Accuracy: 0.9100
Epoch: 800, Test Loss: 0.1979, Test Accuracy: 0.9050, Train Accuracy: 0.9113
Epoch: 900, Test Loss: 0.1854, Test Accuracy: 0.9150, Train Accuracy: 0.9137
Epoch: 999, Test Loss: 0.0108, Test Accuracy: 1.0000, Train Accuracy: 0.9975
Training complete!
Full batch gradient descent
Epoch: 0, Test Loss: 